In [6]:
import sys
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
%reload_ext memory_profiler


In [7]:
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy.metrics import margin, radius, ratio, trace, frobenius
from MKLpy.preprocessing import kernel_normalization
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [8]:
from mkl_data_processing import storage_location,cross_validation_results_location
from mkl_model_cross_validation import load_pickled_in_filename

In [9]:
import gc #for garbage collection

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
import fileutils
import psutil
import pickle
import fnmatch
import os
import torch
import time
import clfutils
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, hamming_loss
import new_alternate_single_svm as nalsvm

In [12]:
def load_pickled_in_filename(file):
    # load a simple pickled file and return it. its a bit different to the method used for the dictionary as this
    # is pure Python 3.x
    pickle_in = open(file, 'rb')
    return pickle.load(pickle_in)


In [13]:
import os
import pickle
file_location = '/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CCL.L'
files_pkl = os.listdir(file_location)

In [14]:
file_to_be_loaded = os.path.join(file_location, files_pkl[3])

In [15]:
load_pickled_in_filename(file_to_be_loaded)

{'20170704': [(1, 100, [1.0, 1.0, 0.32420091324200917])],
 '20170705': [(1,
   100,
   [0.9628099173553719, 0.9813278008298755, 0.8921161825726142])],
 '20170711': [(1,
   100,
   [0.7952127659574468, 0.9533333333333333, 0.9986666666666666])],
 '20170818': [(1,
   100,
   [0.858974358974359, 0.41025641025641024, 0.3137614678899082])]}

In [16]:
 for symbol in ['ECM.L','SHP.L']:
        print(symbol) # which symbol - unnecessary at this point
        cross_validation_data_location = cross_validation_results_location(symbol)
        print(cross_validation_data_location)
        #clean_data_location = storage_location(symbol)

ECM.L
/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/ECM.L
SHP.L
/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/SHP.L


In [17]:
pkl_files_cross_val = os.listdir(cross_validation_data_location)

In [18]:
idx =1
loaded_pickle =load_pickled_in_filename(os.path.join(cross_validation_data_location,pkl_files_cross_val[idx]))

In [22]:
fitted_model_dates = [loaded_pickle.columns.values[f][1] for f,_ in enumerate(loaded_pickle.columns.values)]

In [47]:
loaded_dict= loaded_pickle.to_dict()

In [58]:
keys =list(loaded_dict.keys())

In [61]:
symbol = keys[0][0]
date = keys[0][1]
label_type = keys[0][2]

In [74]:
loaded_pickle[keys[0]][0].values[0][1]

0.9920948616600791

In [28]:
 #{k1 + k2 : v for k1, v1 in d.iteritems() for k2, v in v1.iteritems() } 
example_dict = {k1 + k2 : v for k1, v1 in loaded_pickle.iteritems() for k2, v in v1.iteritems() } 

In [63]:

results_df = pd.DataFrame.from_dict(example_dict )
    


In [33]:
KeysForLabelsFive  = [f for f in list(example_dict.keys()) if str('LabelsAlternateFive') in f]
KeysForLabelsFour  = [f for f in list(example_dict.keys()) if str('LabelsAlternateFour') in f]


In [74]:
ECM_results = [f for f in results_df.columns.values if str('ECM.L') in f]
SHP_results = [f for f in results_df.columns.values if str('SHP.L') in f]


In [86]:
ECM_results_Label4 = [f for f in ECM_results if str('LabelsAlternateFour') in f]
ECM_results_Label5 = [f for f in ECM_results if str('LabelsAlternateFive') in f]

In [90]:
results_df[ECM_results_Label4].mean(axis=0)

ECM.L  20170704  LabelsAlternateFour  0.0  0.01      0.378403
                                           1.00      0.404097
                                           10.00     0.399031
                                           100.00    0.410935
                                      0.1  0.01      0.330443
                                                       ...   
       20180419  LabelsAlternateFour  0.2  100.00    0.692779
                                      1.0  0.01      0.692779
                                           1.00      0.692779
                                           10.00     0.692779
                                           100.00    0.692779
Length: 992, dtype: float64

In [87]:
results_df[ECM_results_Label5].mean(axis=0)

ECM.L  20170705  LabelsAlternateFive  0.0  0.01      0.990793
                                           1.00      0.990793
                                           10.00     0.990793
                                           100.00    0.990793
                                      0.1  0.01      0.990793
                                                       ...   
       20180411  LabelsAlternateFive  0.2  100.00    0.997470
                                      1.0  0.01      0.997470
                                           1.00      0.997470
                                           10.00     0.997470
                                           100.00    0.997470
Length: 928, dtype: float64

In [93]:
dates_ECM_label_5 = [f[1] for f in ECM_results_Label5]

In [96]:
dates = np.unique(dates_ECM_label_5)

array(['20170705', '20170710', '20170711', '20170712', '20170717',
       '20170718', '20170719', '20170720', '20170725', '20170726',
       '20170728', '20170802', '20170803', '20170807', '20170808',
       '20170809', '20170810', '20170811', '20170814', '20170815',
       '20170816', '20170821', '20170822', '20170823', '20170824',
       '20170829', '20170830', '20170901', '20170905', '20170906',
       '20170907', '20170911', '20170912', '20170914', '20170918',
       '20170920', '20170921', '20170922', '20170925', '20170926',
       '20170929', '20171004', '20171009', '20171010', '20171016',
       '20171017', '20171023', '20171024', '20171025', '20171030',
       '20171031', '20180202', '20180206', '20180216', '20180220',
       '20180226', '20180227', '20180411'], dtype='<U8')